In [2]:
import frost2df # for frost.help
from frost2df import frost2df, obs2df, lightning2df, codetable
import datetime
import numpy as np
import folium
from folium.features import DivIcon
from pyproj import CRS
from pyproj import Transformer
import requests
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display


client_id = 'b8b1793b-27ff-4f4d-a081-fcbcc5065b53'
client_secret = '7f24c0ca-ca82-4ed6-afcd-23e657c2e78c'

# Observasjonsdata

In [3]:
# station metadata
frost2df('sources', {'name': '*ålesund*'})

,@type,id,name,shortName,country,countryCode,wmoId,masl,validFrom,county,countyId,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,icaoCodes
0,SensorSystem,SN60945,ÅLESUND IV,Ålesund,Norge,NO,1224.0,15,2009-06-24T00:00:00.000Z,MØRE OG ROMSDAL,15,ÅLESUND,1507,[ÅLESUND KOMMUNE],"[0-20000-0-01224, 10.249.0.115]",0-20000-0-01224,Point,"[6.2108, 62.4703]",False,NaN
1,SensorSystem,SN60947,FV395 ÅLESUND,FV395 Ålesund,Norge,NO,NaN,10,2012-01-01T00:00:00.000Z,MØRE OG ROMSDAL,15,ÅLESUND,1507,[STATENS VEGVESEN],"[139, 1529001]",0-578-0-60947,Point,"[6.1945, 62.4703]",False,NaN
2,SensorSystem,SN99902,NY-ÅLESUND - RABBEN FLYPLASS,Ny-Ålesund FP,Svalbard og Jan Mayen,SJ,NaN,41,1997-07-01T00:00:00.000Z,SVALBARD,21,SVALBARD,2100,[KINGS BAY A/S],[ENAS],0-744-0-99902,Point,"[11.8837, 78.9273]",False,[ENAS]
3,SensorSystem,SN99910,NY-ÅLESUND,Ny-Ålesund,Svalbard og Jan Mayen,SJ,1007.0,8,1974-07-26T00:00:00.000Z,SVALBARD,21,SVALBARD,2100,"[MET.NO, STIFTUNG ALFRED-WEGENER-INSTITUT FUR ...","[0-20000-0-01007, 10.44.20.254, 193.156.10.28]",0-20000-0-01007,Point,"[11.9325, 78.9218]",False,NaN


In [4]:
df_stasjoner = frost2df('sources', {'county': '*møre*'})

df_stasjoner

,@type,id,name,shortName,country,countryCode,masl,validFrom,county,countyId,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId,icaoCodes
0,SensorSystem,SN61065,E136 TRESFJORDBRUA,Tresfjordbrua,Norge,NO,20,2016-04-12T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[STATENS VEGVESEN],"[1529022, 1651]",0-578-0-61065,Point,"[7.1135, 62.6112]",False,NaN,NaN
1,SensorSystem,SN62240,E39 LØNNSET,E39 Lønnset,Norge,NO,15,2014-09-11T00:00:00.000Z,MØRE OG ROMSDAL,15,MOLDE,1506,[STATENS VEGVESEN],"[1529011, 295]",0-578-0-62240,Point,"[7.3702, 62.7603]",False,NaN,NaN
2,SensorSystem,SN60670,FV650 LIABYGDA,FV650 Liabygda,Norge,NO,318,2014-09-11T00:00:00.000Z,MØRE OG ROMSDAL,15,STRANDA,1525,[STATENS VEGVESEN],"[1529015, 297]",0-578-0-60670,Point,"[7.0663, 62.3285]",False,NaN,NaN
3,SensorSystem,SN59800,SVINØY FYR,Svinøy fyr,Norge,NO,38,1955-06-01T00:00:00.000Z,MØRE OG ROMSDAL,15,HERØY,1515,[MET.NO],"[0-20000-0-01205, 10.249.1.56]",0-20000-0-01205,Point,"[5.268, 62.3293]",False,1205.0,NaN
4,SensorSystem,SN59665,VOLDA BRANNSTASJON,Volda brannst.,Norge,NO,50,2019-09-19T00:00:00.000Z,MØRE OG ROMSDAL,15,VOLDA,1577,[VOLDA KOMMUNE],[518101035],0-578-0-59665,Point,"[6.0842, 62.1447]",False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,SensorSystem,SN59815,FV5876 RUNDEBRUA,FV5876 Rundebrua,Norge,NO,20,2020-09-16T00:00:00.000Z,MØRE OG ROMSDAL,15,HERØY,1515,[STATENS VEGVESEN],"[1823, 3000047]",0-578-0-59815,Point,"[5.6275, 62.3792]",False,NaN,NaN
81,SensorSystem,SN60030,SYKKYLVEN - STRAUMGJERDET,Straumgjerdet,Norge,NO,2,2019-10-24T00:00:00.000Z,MØRE OG ROMSDAL,15,SYKKYLVEN,1528,[SYKKYLVEN KOMMUNE],[518101042],0-578-0-60030,Point,"[6.5912, 62.337]",False,NaN,NaN
82,SensorSystem,SN61340,ÅNDALSNES - KAMSHAUGEN,Åndalsnes,Norge,NO,4,2011-11-30T00:00:00.000Z,MØRE OG ROMSDAL,15,RAUMA,1539,[RAUMA KOMMUNE],[1517],0-578-0-61340,Point,"[7.672, 62.5643]",False,NaN,NaN
83,SensorSystem,SN60990,VIGRA,Vigra,Norge,NO,22,1958-07-01T00:00:00.000Z,MØRE OG ROMSDAL,15,GISKE,1532,[AVINOR],"[0-20000-0-01210, 10.249.0.154, 10.249.0.59, E...",0-20000-0-01210,Point,"[6.115, 62.5617]",False,1210.0,[ENAL]


In [5]:
df_stasjoner['validFrom'] = pd.to_datetime(df_stasjoner['validFrom'])

In [6]:
df_stasjoner.dtypes

@type                                object
id                                   object
name                                 object
shortName                            object
country                              object
countryCode                          object
masl                                  int64
validFrom               datetime64[ns, UTC]
county                               object
countyId                              int64
municipality                         object
municipalityId                        int64
stationHolders                       object
externalIds                          object
wigosId                              object
geometry.@type                       object
geometry.coordinates                 object
geometry.nearest                       bool
wmoId                               float64
icaoCodes                            object
dtype: object

In [7]:
df_stasjoner

,@type,id,name,shortName,country,countryCode,masl,validFrom,county,countyId,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId,icaoCodes
0,SensorSystem,SN61065,E136 TRESFJORDBRUA,Tresfjordbrua,Norge,NO,20,2016-04-12 00:00:00+00:00,MØRE OG ROMSDAL,15,VESTNES,1535,[STATENS VEGVESEN],"[1529022, 1651]",0-578-0-61065,Point,"[7.1135, 62.6112]",False,NaN,NaN
1,SensorSystem,SN62240,E39 LØNNSET,E39 Lønnset,Norge,NO,15,2014-09-11 00:00:00+00:00,MØRE OG ROMSDAL,15,MOLDE,1506,[STATENS VEGVESEN],"[1529011, 295]",0-578-0-62240,Point,"[7.3702, 62.7603]",False,NaN,NaN
2,SensorSystem,SN60670,FV650 LIABYGDA,FV650 Liabygda,Norge,NO,318,2014-09-11 00:00:00+00:00,MØRE OG ROMSDAL,15,STRANDA,1525,[STATENS VEGVESEN],"[1529015, 297]",0-578-0-60670,Point,"[7.0663, 62.3285]",False,NaN,NaN
3,SensorSystem,SN59800,SVINØY FYR,Svinøy fyr,Norge,NO,38,1955-06-01 00:00:00+00:00,MØRE OG ROMSDAL,15,HERØY,1515,[MET.NO],"[0-20000-0-01205, 10.249.1.56]",0-20000-0-01205,Point,"[5.268, 62.3293]",False,1205.0,NaN
4,SensorSystem,SN59665,VOLDA BRANNSTASJON,Volda brannst.,Norge,NO,50,2019-09-19 00:00:00+00:00,MØRE OG ROMSDAL,15,VOLDA,1577,[VOLDA KOMMUNE],[518101035],0-578-0-59665,Point,"[6.0842, 62.1447]",False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,SensorSystem,SN59815,FV5876 RUNDEBRUA,FV5876 Rundebrua,Norge,NO,20,2020-09-16 00:00:00+00:00,MØRE OG ROMSDAL,15,HERØY,1515,[STATENS VEGVESEN],"[1823, 3000047]",0-578-0-59815,Point,"[5.6275, 62.3792]",False,NaN,NaN
81,SensorSystem,SN60030,SYKKYLVEN - STRAUMGJERDET,Straumgjerdet,Norge,NO,2,2019-10-24 00:00:00+00:00,MØRE OG ROMSDAL,15,SYKKYLVEN,1528,[SYKKYLVEN KOMMUNE],[518101042],0-578-0-60030,Point,"[6.5912, 62.337]",False,NaN,NaN
82,SensorSystem,SN61340,ÅNDALSNES - KAMSHAUGEN,Åndalsnes,Norge,NO,4,2011-11-30 00:00:00+00:00,MØRE OG ROMSDAL,15,RAUMA,1539,[RAUMA KOMMUNE],[1517],0-578-0-61340,Point,"[7.672, 62.5643]",False,NaN,NaN
83,SensorSystem,SN60990,VIGRA,Vigra,Norge,NO,22,1958-07-01 00:00:00+00:00,MØRE OG ROMSDAL,15,GISKE,1532,[AVINOR],"[0-20000-0-01210, 10.249.0.154, 10.249.0.59, E...",0-20000-0-01210,Point,"[6.115, 62.5617]",False,1210.0,[ENAL]


In [8]:
mask = (df_stasjoner['validFrom'] > '1700-01-01') & (df_stasjoner['validFrom'] <= '1980-01-01')

In [9]:
df_utvalg= df_stasjoner.loc[mask]
df_utvalg.reset_index(inplace=True)

In [10]:
y = df_utvalg['geometry.coordinates'][0][0]
x = df_utvalg['geometry.coordinates'][0][1]
x,y

(62.3293, 5.268)

In [11]:
len(df_utvalg)

24

In [12]:

m = folium.Map(location=[df_utvalg['geometry.coordinates'][0][1], y], zoom_start=8) #Bruker transformerte koordinater (ikkej funne ut korleis bruke UTM med folium)

for i in range(len(df_utvalg)):
    folium.Marker(
        location=[df_utvalg['geometry.coordinates'][i][1], df_utvalg['geometry.coordinates'][i][0]],
        popup=df_utvalg['name'][i] + ' | ' + df_utvalg['id'][i] + ' | ' + str(df_utvalg['validFrom'][i]),
        icon=folium.Icon(color="green")
    ).add_to(m)
#Legger til Norgeskart WMS

folium.raster_layers.WmsTileLayer(
    url='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
    name='Norgeskart',
    fmt='image/png',
    layers='topo4',
    attr=u'<a href="http://www.kartverket.no/">Kartverket</a>',
    transparent=True,
    overlay=True,
    control=True,
    
).add_to(m)

m

In [13]:
idliste = ['SN62290', 'SN62270 ', 'SN60800', 'SN60990']

In [14]:
df = df_stasjoner[df_stasjoner['id'].isin(idliste)]
df.reset_index(inplace=True)
df

,index,@type,id,name,shortName,country,countryCode,masl,validFrom,county,...,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId,icaoCodes
0,22,SensorSystem,SN62290,MOLDE - NØISOMHED,Molde − Nøisomhed,Norge,NO,5,1974-04-04 00:00:00+00:00,MØRE OG ROMSDAL,...,MOLDE,1506,[MOLDE KOMMUNE],[10.249.1.68],0-578-0-62290,Point,"[7.2275, 62.7413]",False,NaN,NaN
1,42,SensorSystem,SN60800,ØRSKOG,Ørskog,Norge,NO,5,1895-07-01 00:00:00+00:00,MØRE OG ROMSDAL,...,ÅLESUND,1507,[MET.NO],[10.249.1.23],0-578-0-60800,Point,"[6.8167, 62.4775]",False,NaN,NaN
2,83,SensorSystem,SN60990,VIGRA,Vigra,Norge,NO,22,1958-07-01 00:00:00+00:00,MØRE OG ROMSDAL,...,GISKE,1532,[AVINOR],"[0-20000-0-01210, 10.249.0.154, 10.249.0.59, E...",0-20000-0-01210,Point,"[6.115, 62.5617]",False,1210.0,[ENAL]


In [15]:

m = folium.Map(location=[df['geometry.coordinates'][0][1], y], zoom_start=8) #Bruker transformerte koordinater (ikkej funne ut korleis bruke UTM med folium)

for i in range(len(df)):
    folium.Marker(
        location=[df['geometry.coordinates'][i][1], df['geometry.coordinates'][i][0]],
        popup=df['name'][i] + ' | ' + df['id'][i] + ' | ' + str(df['validFrom'][i]),
        icon=folium.Icon(color="green")
    ).add_to(m)
#Legger til Norgeskart WMS
folium.raster_layers.WmsTileLayer(
    url='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
    name='Norgeskart',
    fmt='image/png',
    layers='topo4',
    attr=u'<a href="http://www.kartverket.no/">Kartverket</a>',
    transparent=True,
    overlay=True,
    control=True,
    
).add_to(m)

m

In [16]:
for i in idliste:
    parameters = {
        'sources': i,
        'elements': 'mean(air_temperature P1D)',
        'referencetime': '2014-04-01/2024-04-03',
        'timeoffsets': 'PT6H'
    }
    display(obs2df(parameters=parameters, verbose=True))

https://frost.met.no/observations/v0.jsonld?sources=SN62290&elements=mean%28air_temperature+P1D%29&referencetime=2014-04-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN62290:0,mean(air_temperature P1D),2016-05-14 00:00:00+00:00,4.1,degC,2,PT6H,P1D,2
1,SN62290:0,mean(air_temperature P1D),2016-05-15 00:00:00+00:00,5.4,degC,2,PT6H,P1D,2
2,SN62290:0,mean(air_temperature P1D),2016-05-16 00:00:00+00:00,5.6,degC,2,PT6H,P1D,2
3,SN62290:0,mean(air_temperature P1D),2016-05-17 00:00:00+00:00,7.7,degC,2,PT6H,P1D,2
4,SN62290:0,mean(air_temperature P1D),2016-05-18 00:00:00+00:00,5.9,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
1652,SN62290:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.8,degC,2,PT6H,P1D,2
1653,SN62290:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,2.2,degC,2,PT6H,P1D,2
1654,SN62290:0,mean(air_temperature P1D),2022-04-04 00:00:00+00:00,1.4,degC,2,PT6H,P1D,2
1655,SN62290:0,mean(air_temperature P1D),2022-04-05 00:00:00+00:00,-0.1,degC,2,PT6H,P1D,2


https://frost.met.no/observations/v0.jsonld?sources=SN62270+&elements=mean%28air_temperature+P1D%29&referencetime=2014-04-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN62270:0,mean(air_temperature P1D),2014-04-01 00:00:00+00:00,3.9,degC,2,PT6H,P1D,2
1,SN62270:0,mean(air_temperature P1D),2014-04-02 00:00:00+00:00,3.6,degC,2,PT6H,P1D,2
2,SN62270:0,mean(air_temperature P1D),2014-04-03 00:00:00+00:00,4.0,degC,2,PT6H,P1D,2
3,SN62270:0,mean(air_temperature P1D),2014-04-04 00:00:00+00:00,3.8,degC,2,PT6H,P1D,2
4,SN62270:0,mean(air_temperature P1D),2014-04-05 00:00:00+00:00,5.2,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
2921,SN62270:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.9,degC,2,PT6H,P1D,2
2922,SN62270:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,1.8,degC,2,PT6H,P1D,2
2923,SN62270:0,mean(air_temperature P1D),2022-04-04 00:00:00+00:00,1.3,degC,2,PT6H,P1D,2
2924,SN62270:0,mean(air_temperature P1D),2022-04-05 00:00:00+00:00,-0.2,degC,2,PT6H,P1D,2


https://frost.met.no/observations/v0.jsonld?sources=SN60800&elements=mean%28air_temperature+P1D%29&referencetime=2014-04-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN60800:0,mean(air_temperature P1D),2014-11-19 00:00:00+00:00,3.1,degC,2,PT6H,P1D,2
1,SN60800:0,mean(air_temperature P1D),2014-11-20 00:00:00+00:00,3.5,degC,2,PT6H,P1D,2
2,SN60800:0,mean(air_temperature P1D),2014-11-21 00:00:00+00:00,3.0,degC,2,PT6H,P1D,2
3,SN60800:0,mean(air_temperature P1D),2014-11-22 00:00:00+00:00,0.1,degC,2,PT6H,P1D,2
4,SN60800:0,mean(air_temperature P1D),2014-11-23 00:00:00+00:00,1.8,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
2689,SN60800:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.1,degC,2,PT6H,P1D,2
2690,SN60800:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,1.2,degC,2,PT6H,P1D,2
2691,SN60800:0,mean(air_temperature P1D),2022-04-04 00:00:00+00:00,1.1,degC,2,PT6H,P1D,2
2692,SN60800:0,mean(air_temperature P1D),2022-04-05 00:00:00+00:00,-0.1,degC,2,PT6H,P1D,2


https://frost.met.no/observations/v0.jsonld?sources=SN60990&elements=mean%28air_temperature+P1D%29&referencetime=2014-04-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN60990:0,mean(air_temperature P1D),2014-04-01 00:00:00+00:00,4.3,degC,2,PT6H,P1D,2
1,SN60990:0,mean(air_temperature P1D),2014-04-02 00:00:00+00:00,3.6,degC,2,PT6H,P1D,2
2,SN60990:0,mean(air_temperature P1D),2014-04-03 00:00:00+00:00,4.7,degC,2,PT6H,P1D,2
3,SN60990:0,mean(air_temperature P1D),2014-04-04 00:00:00+00:00,4.3,degC,2,PT6H,P1D,2
4,SN60990:0,mean(air_temperature P1D),2014-04-05 00:00:00+00:00,6.4,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
2916,SN60990:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.9,degC,2,PT6H,P1D,2
2917,SN60990:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,2.0,degC,2,PT6H,P1D,2
2918,SN60990:0,mean(air_temperature P1D),2022-04-04 00:00:00+00:00,1.7,degC,2,PT6H,P1D,2
2919,SN60990:0,mean(air_temperature P1D),2022-04-05 00:00:00+00:00,0.3,degC,2,PT6H,P1D,2


In [17]:
frost2df('sources', {'municipality': 'Vestnes'})

,@type,id,name,shortName,country,countryCode,masl,validFrom,county,countyId,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId
0,SensorSystem,SN61065,E136 TRESFJORDBRUA,Tresfjordbrua,Norge,NO,20,2016-04-12T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[STATENS VEGVESEN],"[1529022, 1651]",0-578-0-61065,Point,"[7.1135, 62.6112]",False,NaN
1,SensorSystem,SN61064,VESTNES - RÅDHUSET,Vestnes,Norge,NO,30,2019-10-23T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[VESTNES KOMMUNE],[518101038],0-578-0-61064,Point,"[7.0903, 62.6213]",False,NaN
2,SensorSystem,SN61070,E39 ØRSKOGFJELL,E39 Ørskogfjell,Norge,NO,305,2011-01-01T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[STATENS VEGVESEN],"[136, 1529008]",0-578-0-61070,Point,"[6.949, 62.5325]",False,NaN
3,SensorSystem,SN61090,TRESFJORDEN,Tresfjorden,Norge,NO,2,2019-10-23T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[VESTNES KOMMUNE],[518101040],0-578-0-61090,Point,"[7.1307, 62.5247]",False,NaN
4,SensorSystem,SN61060,REKDAL,Rekdal,Norge,NO,43,2008-05-26T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[MET.NO],"[0-20000-0-01214, 10.249.0.163]",0-20000-0-01214,Point,"[6.7552, 62.6512]",False,1214.0
5,SensorSystem,SN61062,TOMREFJORD,Tomrefjord,Norge,NO,3,2019-10-23T00:00:00.000Z,MØRE OG ROMSDAL,15,VESTNES,1535,[VESTNES KOMMUNE],[518101039],0-578-0-61062,Point,"[6.9247, 62.6062]",False,NaN


In [18]:
frost2df('observations/availableTimeSeries', {'sources': 'SN61065', 'timeresolutions' : 'P1D'})

,sourceId,validFrom,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,validTo
0,SN61065:0,2016-04-19T00:00:00.000Z,PT0H,P1D,0,mean(air_temperature P1D),degC,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
1,SN61065:0,2016-04-19T00:00:00.000Z,PT0H,P1D,0,integral_of_deficit(mean(air_temperature P1D) ...,degree-day,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
2,SN61065:0,2016-04-18T00:00:00.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,2021-09-30T00:00:00.000Z
3,SN61065:0,2016-10-26T00:00:00.000Z,PT0H,P1D,0,mean(dew_point_temperature P1D),degC,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,2017-02-28T00:00:00.000Z
4,SN61065:0,2016-04-19T00:00:00.000Z,PT0H,P1D,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
5,SN61065:0,2016-04-19T00:00:00.000Z,PT0H,P1D,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,C,4,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN


In [38]:
#pd.set_option('display.max_rows',500)
for i in idliste:
    display(frost2df('observations/availableTimeSeries', {'sources': i}))

,sourceId,validFrom,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,validTo,level.levelType,level.unit,level.value,codeTable
0,SN62290:0,2016-06-01T00:00:00.000Z,PT0H,P1M,0,over_time(time_of_minimum_air_temperature P1M),Date,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
1,SN62290:0,2016-06-01T00:00:00.000Z,PT0H,P3M,0,mean(air_temperature_anomaly P3M 1961_1990),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2020-12-31T00:00:00.000Z,NaN,NaN,NaN,NaN
2,SN62290:0,1979-10-01T00:00:00.000Z,PT6H,P1Y,0,sum(precipitation_amount P1Y),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2020-12-31T00:00:00.000Z,NaN,NaN,NaN,NaN
3,SN62290:0,2016-06-01T00:00:00.000Z,PT18H,P1M,0,mean(max(air_temperature P1D) P1M),degC,E,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
4,SN62290:0,2016-06-01T00:00:00.000Z,PT18H,P1M,0,max(air_temperature P1M),degC,E,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
5,SN62290:0,2016-05-13T00:00:00.000Z,PT0H,P1D,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,E,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
6,SN62290:0,1979-09-30T00:00:00.000Z,PT0H,PT6H,0,over_time(weather_class1_secondary_significanc...,code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,1986-12-31T00:00:00.000Z,NaN,NaN,NaN,weather_class1_intensity
7,SN62290:0,1974-04-03T00:00:00.000Z,PT6H,P1D,0,sum(precipitation_amount P1D),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
8,SN62290:0,2016-05-13T00:00:00.000Z,PT18H,P1D,0,sum(precipitation_amount P1D),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
9,SN62290:0,2016-06-01T00:00:00.000Z,PT18H,P3M,0,min(air_temperature P3M),degC,E,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2020-12-31T00:00:00.000Z,height_above_ground,m,2.0,NaN


,sourceId,validFrom,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,codeTable,validTo
0,SN62270:0,1995-01-01T07:50:00.000Z,PT20M,PT30M,0,cloud_area_fraction2,code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
1,SN62270:0,2017-08-01T00:00:00.000Z,PT0H,P1M,0,best_estimate_mean(air_temperature_anomaly P1M...,degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
2,SN62270:0,2017-08-01T00:00:00.000Z,PT0H,P1M,0,best_estimate_mean(air_temperature P1M),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN,NaN
3,SN62270:0,2006-05-01T00:00:00.000Z,PT0H,P1M,0,mean_k(air_temperature P1M),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN,NaN
4,SN62270:0,2002-10-01T00:00:00.000Z,PT18H,P1M,0,mean(max(wind_speed P1D) P1M),m/s,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,10.0,NaN,NaN
5,SN62270:0,2003-01-01T00:00:00.000Z,PT0H,P1M,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN,NaN
6,SN62270:0,2004-11-03T00:00:00.000Z,PT18H,P1D,0,max(wind_speed_of_gust P1D),m/s,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,10.0,NaN,NaN
7,SN62270:0,2002-09-08T00:00:00.000Z,PT18H,P1D,0,mean(max(wind_speed PT1H) P1D),m/s,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,10.0,NaN,NaN
8,SN62270:0,1997-04-22T08:20:00.000Z,PT20M,PT30M,0,visibility_in_air_vertical_direction,meters,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
9,SN62270:0,2008-07-30T05:50:00.000Z,PT20M,PT30M,0,visibility_in_air_rvr_validation_x1r,NaN,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,metar_rvr_validation,NaN


,sourceId,validFrom,validTo,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,codeTable
0,SN60800:0,2014-12-01T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT0H,P1M,0,mean(air_temperature P1M),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
1,SN60800:0,2020-06-15T00:00:00.000Z,NaN,PT0H,P1M,0,mean(air_temperature P1M),degC,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
2,SN60800:0,2014-12-01T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT6H,P1M,0,mean(air_temperature P1M),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
3,SN60800:0,2020-06-15T00:00:00.000Z,NaN,PT6H,P1M,0,mean(air_temperature P1M),degC,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
4,SN60800:0,1957-01-01T00:00:00.000Z,2013-01-31T00:00:00.000Z,PT6H,P1D,0,snow_coverage_type,code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,snow_cover_code
5,SN60800:0,1895-09-01T00:00:00.000Z,NaN,PT6H,P6M,0,sum(precipitation_amount_anomaly P6M 1991_2020),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
6,SN60800:0,2014-11-17T00:00:00.000Z,2017-06-06T08:16:37.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,A,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
7,SN60800:0,2017-06-06T08:16:37.000Z,2017-06-06T08:29:27.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
8,SN60800:0,2017-06-06T08:29:27.000Z,2020-06-15T00:00:00.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,A,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
9,SN60800:0,2020-06-15T00:00:00.000Z,2020-12-07T07:51:03.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,A,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN


,sourceId,validFrom,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,validTo,level.levelType,level.unit,level.value,codeTable
0,SN60990:0,1958-07-01T00:00:00.000Z,PT0H,P1M,0,mean(water_vapor_partial_pressure_in_air P1M),hPa,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN,NaN
1,SN60990:0,1958-09-01T00:00:00.000Z,PT18H,P1Y,0,min(air_temperature P1Y),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2004-09-14T00:00:00.000Z,height_above_ground,m,2.0,NaN
2,SN60990:0,2004-09-14T00:00:00.000Z,PT18H,P1Y,0,min(air_temperature P1Y),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
3,SN60990:0,1958-09-01T00:00:00.000Z,PT0H,P1Y,0,integral_of_deficit(mean(air_temperature P1D) ...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2001-04-11T03:00:00.000Z,height_above_ground,m,2.0,NaN
4,SN60990:0,2001-04-11T03:00:00.000Z,PT0H,P1Y,0,integral_of_deficit(mean(air_temperature P1D) ...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2004-09-07T21:00:00.000Z,height_above_ground,m,2.0,NaN
5,SN60990:0,2004-09-07T21:00:00.000Z,PT0H,P1Y,0,integral_of_deficit(mean(air_temperature P1D) ...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
6,SN60990:0,1958-08-01T00:00:00.000Z,PT0H,PT6H,0,surface_air_pressure,hPa,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2004-09-14T00:00:00.000Z,height_above_ground,m,2.0,NaN
7,SN60990:0,2004-09-14T00:00:00.000Z,PT0H,PT1H,0,surface_air_pressure,hPa,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,2.0,NaN
8,SN60990:0,1958-07-01T00:00:00.000Z,PT18H,P1D,0,max(max(wind_speed PT1H) P1D),m/s,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,height_above_ground,m,10.0,NaN
9,SN60990:0,1958-07-01T00:00:00.000Z,PT0H,P1D,0,boolean_fair_weather(cloud_area_fraction P1D),sum,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,2020-08-31T00:00:00.000Z,NaN,NaN,NaN,NaN


In [20]:
# observations - simple table
parameters = {
    'sources': 'SN90450',
    'elements': 'sum(precipitation_amount P1D)',
    'timeoffsets': 'PT6H',
    'referencetime': '1900-03-09/2018-05-17',
}
obs2df(parameters=parameters)

,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution
0,SN90450:0,sum(precipitation_amount P1D),1920-07-02 00:00:00+00:00,4.8,mm,2,PT6H,P1D
1,SN90450:0,sum(precipitation_amount P1D),1920-07-03 00:00:00+00:00,0.0,mm,2,PT6H,P1D
2,SN90450:0,sum(precipitation_amount P1D),1920-07-04 00:00:00+00:00,7.2,mm,2,PT6H,P1D
3,SN90450:0,sum(precipitation_amount P1D),1920-07-05 00:00:00+00:00,0.0,mm,2,PT6H,P1D
4,SN90450:0,sum(precipitation_amount P1D),1920-07-06 00:00:00+00:00,0.0,mm,2,PT6H,P1D
...,...,...,...,...,...,...,...,...
35743,SN90450:0,sum(precipitation_amount P1D),2018-05-12 00:00:00+00:00,1.9,mm,0,PT6H,P1D
35744,SN90450:0,sum(precipitation_amount P1D),2018-05-13 00:00:00+00:00,0.0,mm,0,PT6H,P1D
35745,SN90450:0,sum(precipitation_amount P1D),2018-05-14 00:00:00+00:00,0.0,mm,0,PT6H,P1D
35746,SN90450:0,sum(precipitation_amount P1D),2018-05-15 00:00:00+00:00,0.0,mm,0,PT6H,P1D


In [21]:
# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '2010-04-01/2020-04-03',
}
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()
# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])
#Note: the block below is tailored to the observations endpoint, and changes would need to be made to accomodate other endpoints.

# This will return a Dataframe with all of the observations in a table format
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df = df.append(row)

df_par = df.reset_index()
df.head()

Data retrieved from frost.met.no!


C:\Users\jan.aalbu\AppData\Local\Temp\ipykernel_15176\2919579314.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)
C:\Users\jan.aalbu\AppData\Local\Temp\ipykernel_15176\2919579314.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)
C:\Users\jan.aalbu\AppData\Local\Temp\ipykernel_15176\2919579314.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)
C:\Users\jan.aalbu\AppData\Local\Temp\ipykernel_15176\2919579314.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)
C:\Users\jan.aalbu\AppData\Local\Temp\ipykernel_15176\2919579314.py:28: FutureWarning: The f

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,mean(air_temperature P1D),3.2,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
1,mean(air_temperature P1D),3.0,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT6H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
2,sum(precipitation_amount P1D),13.5,mm,NaN,PT18H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
3,sum(precipitation_amount P1D),29.0,mm,NaN,PT6H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
4,mean(wind_speed P1D),1.7,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0


In [22]:
df_par

,index,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,0,mean(air_temperature P1D),3.2,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
1,1,mean(air_temperature P1D),3.0,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT6H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
2,2,sum(precipitation_amount P1D),13.5,mm,NaN,PT18H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
3,3,sum(precipitation_amount P1D),29.0,mm,NaN,PT6H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
4,4,mean(wind_speed P1D),1.7,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36537,0,mean(air_temperature P1D),-5.7,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,0.0,2020-04-02T00:00:00.000Z,SN90450:0
36538,1,mean(air_temperature P1D),-4.4,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT6H,P1D,0,C,2,2.0,2020-04-02T00:00:00.000Z,SN90450:0
36539,2,sum(precipitation_amount P1D),0.0,mm,NaN,PT18H,P1D,0,C,2,2.0,2020-04-02T00:00:00.000Z,SN90450:0
36540,3,sum(precipitation_amount P1D),1.4,mm,NaN,PT6H,P1D,0,C,2,0.0,2020-04-02T00:00:00.000Z,SN90450:0


In [23]:
df_utvalg = df_par.loc[df_par['elementId'] == 'mean(wind_speed P1D)']
df_utvalg

,index,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
4,4,mean(wind_speed P1D),1.7,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-01T00:00:00.000Z,SN18700:0
9,4,mean(wind_speed P1D),2.9,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-02T00:00:00.000Z,SN18700:0
14,4,mean(wind_speed P1D),2.3,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-03T00:00:00.000Z,SN18700:0
19,4,mean(wind_speed P1D),3.6,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-04T00:00:00.000Z,SN18700:0
24,4,mean(wind_speed P1D),2.4,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2010-04-05T00:00:00.000Z,SN18700:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36521,4,mean(wind_speed P1D),2.9,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2020-03-29T00:00:00.000Z,SN90450:0
36526,4,mean(wind_speed P1D),3.8,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2020-03-30T00:00:00.000Z,SN90450:0
36531,4,mean(wind_speed P1D),6.8,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2020-03-31T00:00:00.000Z,SN90450:0
36536,4,mean(wind_speed P1D),1.8,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2020-04-01T00:00:00.000Z,SN90450:0


In [24]:
# observations - recurring
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '2014-04-01/2024-04-03',
}
obs2df(parameters=parameters, verbose=True)

https://frost.met.no/observations/v0.jsonld?sources=SN18700%2CSN90450&elements=mean%28air_temperature+P1D%29%2Csum%28precipitation_amount+P1D%29%2Cmean%28wind_speed+P1D%29&referencetime=2014-04-01%2F2024-04-03


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN18700:0,mean(air_temperature P1D),2014-04-01 00:00:00+00:00,4.0,degC,0.0,PT0H,P1D,2.0
1,SN18700:0,mean(air_temperature P1D),2014-04-01 00:00:00+00:00,5.1,degC,2.0,PT6H,P1D,2.0
2,SN18700:0,sum(precipitation_amount P1D),2014-04-01 00:00:00+00:00,0.0,mm,2.0,PT18H,P1D,NaN
3,SN18700:0,sum(precipitation_amount P1D),2014-04-01 00:00:00+00:00,0.0,mm,2.0,PT6H,P1D,NaN
4,SN18700:0,mean(wind_speed P1D),2014-04-01 00:00:00+00:00,2.0,m/s,2.0,PT0H,P1D,10.0
...,...,...,...,...,...,...,...,...,...
29264,SN90450:0,sum(precipitation_amount P1D),2022-04-05 00:00:00+00:00,0.7,mm,NaN,PT18H,P1D,NaN
29265,SN90450:0,sum(precipitation_amount P1D),2022-04-05 00:00:00+00:00,0.0,mm,0.0,PT6H,P1D,NaN
29266,SN90450:0,mean(wind_speed P1D),2022-04-05 00:00:00+00:00,5.0,m/s,2.0,PT0H,P1D,10.0
29267,SN90450:0,mean(air_temperature P1D),2022-04-06 00:00:00+00:00,-1.9,degC,NaN,PT6H,P1D,2.0


In [25]:
# observations - simple toplist
parameters = {
    'sources': 'SN90450',
    'elements': 'sum(precipitation_amount P1D)',
    'timeoffsets': 'PT6H',
    'referencetime': '2018-03-09/2018-05-17',
}
df=frost2df('observations', parameters)
fav_cols = ['sourceId','elementId','referenceTime','value','unit','qualityCode','timeOffset','timeResolution']
df[fav_cols].nlargest(columns='value', n=5)

,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution
9,SN90450:0,sum(precipitation_amount P1D),2018-03-18T00:00:00.000Z,20.2,mm,0,PT6H,P1D
10,SN90450:0,sum(precipitation_amount P1D),2018-03-19T00:00:00.000Z,15.4,mm,0,PT6H,P1D
20,SN90450:0,sum(precipitation_amount P1D),2018-03-29T00:00:00.000Z,12.9,mm,0,PT6H,P1D
30,SN90450:0,sum(precipitation_amount P1D),2018-04-08T00:00:00.000Z,9.8,mm,0,PT6H,P1D
18,SN90450:0,sum(precipitation_amount P1D),2018-03-27T00:00:00.000Z,8.5,mm,0,PT6H,P1D


In [26]:
# county records
frost2df('records/countyExtremes', {'counties': 'Møre og Romsdal'})

,sourceId,sourceName,county,municipality,elementId,month,referenceTime,value
0,SN63420,Sunndalsøra Iii,Møre Og Romsdal,Sunndal,max(air_temperature P1D),1,2020-01-02,19.0
1,SN63420,Sunndalsøra Iii,Møre Og Romsdal,Sunndal,max(air_temperature P1D),2,1990-02-23,18.9
2,SN63500,Sunndal,Møre Og Romsdal,Sunndal,max(air_temperature P1D),3,1961-03-09,18.5
3,SN64550,Tingvoll - Hanem,Møre Og Romsdal,Tingvoll,max(air_temperature P1D),4,2000-04-30,23.8
4,SN61420,Marstein,Møre Og Romsdal,Rauma,max(air_temperature P1D),5,2018-05-30,30.0
5,SN60500,Tafjord,Møre Og Romsdal,Norddal,max(air_temperature P1D),6,2020-06-27,31.8
6,SN60890,Brusdalsvatn Ii,Møre Og Romsdal,Ålesund,max(air_temperature P1D),7,2014-07-09,34.0
7,SN63500,Sunndal,Møre Og Romsdal,Sunndal,max(air_temperature P1D),8,1969-08-01,32.0
8,SN63420,Sunndalsøra Iii,Møre Og Romsdal,Sunndal,max(air_temperature P1D),9,1999-09-03,27.6
9,SN62270,Molde Lufthavn,Møre Og Romsdal,Molde,max(air_temperature P1D),10,2005-10-11,25.6


# Elementer

In [27]:
# element metadata
frost2df('elements', {'ids': 'over_time(precipitation_type_automatic P1D)'}).T

,0
id,over_time(precipitation_type_automatic P1D)
name,Precipitation type automatic WMO
description,Summary of precipitation types reported automa...
unit,code
codeTable,weather_class1_significance
status,METNO-name
category,Precipitation
calculationMethod.baseName,precipitation_type_automatic
calculationMethod.method,over_time
calculationMethod.period,P1D


In [28]:
# kodetabell (id, element_id, oldcode)
codetable(oldcode='SLAGWA')

https://frost.met.no/elements/v0.jsonld?oldElementCodes=SLAGWA

https://frost.met.no/elements/codeTables/v0.jsonld?ids=weather_class1_significance&lang=nb-NO
Været, hovedsaklig nedbørtyper, ved observasjonstidspunktet for SYNOP, eller siden siste observasjon for nedbørstasjoner. Rapporteres etter signifikans.
Tabell som følger SYNOP/BUFR rapportering
32 elements


,value,description,additionalInfo
0,-1,.,.
1,1,Sludd,SL
2,2,Snø,SS
3,3,Regn,RR
4,4,Sluddbyge,LB
5,5,Snøbyge,SB
6,6,Kornsnø,KS
7,7,Regnbyge,RB
8,8,Yr,YR
9,9,Sprøhagl,SH


# IVF-data

In [29]:
# IDF data
parameters = {
    'sources': 'SN18701',
    'unit': 'mm',
    'fields': 'values'
}
frost2df('frequencies/rainfall', parameters).T

,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
intensity,8.6472,9.4086,5.634,7.53,8.784,9.987,10.368,11.544,12.711,13.875,...,5.1756,5.7228,6.2676,6.81,4.032,5.2668,6.0858,6.8706,7.119,7.8858
duration,3,3,5,5,5,5,5,5,5,5,...,2,2,2,2,3,3,3,3,3,3
frequency,100,200,2,5,10,20,25,50,100,200,...,25,50,100,200,2,5,10,20,25,50
sourceId,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,...,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701,SN18701


In [30]:
# available IDF series
frost2df('frequencies/rainfall/availableSources')

,sourceId,validFrom,validTo,numberOfSeasons,firstYearOfPeriod,lastYearOfPeriod,version
0,SN1220,1974-04-24T00:00:00Z,1995-12-05T00:00:00Z,18.0,1974.0,1995.0,NaN
1,SN2840,1972-05-30T00:00:00Z,1987-11-16T00:00:00Z,16.0,1972.0,1987.0,NaN
2,SN3030,1970-05-01T00:00:00Z,2016-07-31T00:00:00Z,30.0,1970.0,2013.0,NaN
3,SN3120,2003-04-01T00:00:00Z,2017-03-31T00:00:00Z,11.0,2003.0,2013.0,NaN
4,SN3810,1968-01-01T00:00:00Z,2021-04-18T00:00:00Z,42.0,1968.0,2017.0,NaN
5,SN4220,1975-06-10T00:00:00Z,1992-08-28T00:00:00Z,15.0,1975.0,1992.0,NaN
6,SN4781,1967-05-11T00:00:00Z,2010-08-29T00:00:00Z,43.0,1967.0,2010.0,NaN
7,SN11620,1974-08-02T00:00:00Z,1995-10-27T00:00:00Z,21.0,1974.0,1996.0,NaN
8,SN12290,1968-05-01T00:00:00Z,2021-04-20T00:00:00Z,45.0,1968.0,2017.0,NaN
9,SN12670,1969-07-07T00:00:00Z,1991-09-24T00:00:00Z,23.0,1969.0,1991.0,NaN


# Lyndata

In [31]:
# for polygon box, use: https://boundingbox.klokantech.com/
# both functions fail when no data
# can only fetch one day at a time now, which is kind of useless
parameters = {
    'referencetime': '2020-01-24/2020-01-25',
    'geometry': 'POLYGON((8.229474 63.530051, 9.122113 63.530051, 9.122113 63.265625, 8.229474 63.265625, 8.229474 63.530051))'
}
lightning2df(parameters)

,versionNumber,year,month,day,hour,minute,second,nanosecond,width,length,...,lengthofSemiMajorAxis,lengthofSemiMinorAxis,chiSquareValue,risetimeOfWaveform(ms),peakToZeroTime,maxRateOfRise,cloudIndicator,angleIndicator,signalIndicator,timingIndicator
0,0,2020,1,24,7,38,12,92530944,63.4664,8.6611,...,0.80,0.4,0.18,0.0,0.0,0.0,1,1,0,1
1,0,2020,1,24,7,38,12,92607744,63.4620,8.6619,...,0.89,0.4,0.21,13.2,4.8,0.0,1,1,0,1
2,0,2020,1,24,7,38,12,136594688,63.4389,8.6917,...,0.40,0.4,1.81,10.3,8.8,3.6,1,1,0,1
3,0,2020,1,24,7,35,22,834980608,63.4533,8.5758,...,0.48,0.4,0.31,0.0,0.0,0.0,1,1,0,1
4,0,2020,1,24,7,35,22,846101504,63.4147,8.5893,...,0.40,0.4,1.31,5.2,7.3,1.5,1,1,0,1
5,0,2020,1,24,7,35,22,834732800,63.4129,8.5762,...,0.40,0.4,0.07,0.0,0.0,0.0,1,1,0,1
6,0,2020,1,24,7,35,22,834777088,63.4500,8.5816,...,0.40,0.4,2.22,0.0,0.0,0.0,1,1,0,1
7,0,2020,1,24,7,35,22,835043328,63.3203,8.3598,...,0.95,0.4,0.37,0.0,0.0,0.0,1,1,0,1
8,0,2020,1,24,7,35,22,835086336,63.4338,8.5815,...,0.48,0.4,0.25,0.0,0.0,0.0,1,1,0,1
9,0,2020,1,24,7,33,27,466408704,63.4022,8.3919,...,0.40,0.4,4.90,19.6,9.3,4.6,1,1,0,1


# Klimanormaler

In [32]:
# climate normals
parameters = {
    'sources': 'SN64700',
    'elements': 'mean(air_temperature P1Y),sum(precipitation_amount P1Y)',
    'period': '1991/2020'
}
frost2df('climatenormals', parameters)

,sourceId,elementId,period,normal
0,SN64700,sum(precipitation_amount P1Y),1991/2020,1739.0
1,SN64700,mean(air_temperature P1Y),1991/2020,4.8


In [33]:
for i in idliste:
    parameters = {
    'sources': i,
    'elements': 'mean(air_temperature P1Y),sum(precipitation_amount P1Y)',
    'period': '1991/2020'
    }
    
    print(frost2df('climatenormals', parameters))

  sourceId                      elementId     period  normal
0  SN62290  sum(precipitation_amount P1Y)  1991/2020    1669
  sourceId                  elementId     period  normal
0  SN62270  mean(air_temperature P1Y)  1991/2020     7.1
  sourceId                      elementId     period  normal
0  SN60800  sum(precipitation_amount P1Y)  1991/2020  1539.0
1  SN60800      mean(air_temperature P1Y)  1991/2020     7.4
  sourceId                      elementId     period  normal
0  SN60990  sum(precipitation_amount P1Y)  1991/2020  1451.0
1  SN60990      mean(air_temperature P1Y)  1991/2020     7.9


In [34]:
# available climate normals
frost2df('climatenormals/available', {'sources': 'SN64700'})

,sourceId,elementId,period
0,SN64700,mean(air_temperature P1D),1991/2020
1,SN64700,mean(air_temperature P1M),1991/2020
2,SN64700,mean(air_temperature P1Y),1991/2020
3,SN64700,number_of_days_gte(sum(precipitation_amount P1...,1961/1990
4,SN64700,number_of_days_gte(sum(precipitation_amount P1...,1961/1990
5,SN64700,sum(precipitation_amount P1M),1931/1960
6,SN64700,sum(precipitation_amount P1M),1961/1990
7,SN64700,sum(precipitation_amount P1M),1991/2020
8,SN64700,sum(precipitation_amount P1Y),1931/1960
9,SN64700,sum(precipitation_amount P1Y),1961/1990


# Kvalitetsinfo

In [35]:
# quality code table
frost2df('observations/availableQualityCodes')

,value,shortMeaning,meaning
0,-2,,Reserved - not in use.
1,0,OK,Value is controlled and found O.K.
2,1,OK,"Value is controlled and corrected, or value is..."
3,2,Slightly uncertain,Value is not controlled.
4,3,Slightly uncertain,Reserved - not in use.
5,4,Slightly uncertain,Value is slightly uncertain (not corrected).
6,5,Very uncertain,Value is very uncertain (not corrected).
7,6,"Very uncertain, modeldata","Value is controlled and corrected, or value is..."
8,7,Erroneous,Value is erroneous (not corrected).


In [36]:
# detailed 5-digit quality code description
frost2df('observations/quality', {'flags':'99999'})

,controlType,value,meaning
0,Control level passed,9,Information about level of control is not given
1,The original value deviate from normed observa...,9,Status information is not given
2,Qualitylevel for the original value,9,Quality information is not given
3,Original value corrected,9,Quality information not given
4,Most important quality control method,9,Quality information not given


# Help info

In [37]:
params, responses = frost.help('observations')
params
#responses

NameError: name 'frost' is not defined